In [1]:
from transformers import pipeline
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset
import torch
from sklearn.metrics import classification_report

#Seeding for deterministic results i.e. showing same output 
RANDOM_SEED = 64
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
   torch.cuda.manual_seed(RANDOM_SEED)
   torch.cuda.manual_seed_all(RANDOM_SEED) 
   torch.backends.cudnn.deterministic = True  
   torch.backends.cudnn.benchmark = False


function_mapping = {'OTHER': ['joy', 'love', 'surprise'],
           'NOT_INTERESTED': [''], 
           'DISLIKE':[''], 
           'NOT_CORRECT': [''], 
           'PESSIMISTIC':['sadness'], 
           'WORRIED':['fear'], 
           'ANGRY': ['anger'], 
           'DISAPPOINTED': [''], 
           'BORED': [''], 
           'NOT_APPROVE':[''], 
           'NOT_IMPORTANT': [''], 
           'DISAGREE': [''], 
           'WARN': [''], 
           'COMPLAIN': [''], 
           'THREATEN': [''], 
           'UNWILLING': [''], 
           'DISTRUST' : [''],
           'REFUSE': [''] }

emotion = load_dataset('emotion')
mapping = {
    0:"sadness",
    1:"joy",
    2:"love",
    3:"anger",
    4:"fear",
    5:"surprise"
}
emotion = emotion.map(lambda example: {'emotion': mapping[example['label']]}, 
                      remove_columns=['label'])
def map_labels(batch):
    batch['function'] = [[] for _ in range(len(batch['text']))]
    for i, item in enumerate(batch["function"]):
        for key, value in function_mapping.items():
            for emotion in value:
                if emotion == batch["emotion"][i]:
                    batch["function"][i] = key
                    break
    return batch
emotion = emotion.map(map_labels, batched=True) 

/home/njfernandez/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: emotion/split
Found cached dataset emotion (/home/njfernandez/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)
100%|██████████| 3/3 [00:00<00:00, 727.00it/s]
Loading cached processed dataset at /home/njfernandez/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-da33b75eb48814c5.arrow
Loading cached processed dataset at /home/njfernandez/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-c9fa37d993431ea5.arrow
Loading cached processed dataset at /home/njfernandez/.cache/huggingface/datasets/emotion/s

In [2]:
zero_shot_model='MoritzLaurer/mDeBERTa-v3-base-mnli-xnli'
candidate_labels= {#"lack of interest": "NOT_INTERESTED",
                   #"boredom": "BORED",
                   #"disgust": "DISLIKE", 
                   #"incorrectness": "NOT_CORRECT", 
                   "sadness": "PESSIMISTIC", 
                   "fear": "WORRIED", 
                   "anger": "ANGRY", 
                   #"disappointment": "DISAPPOINTED",
                   #"complaint": "COMPLAIN",
                   #"disapproval": "NOT_APPROVE",
                   #"unimportance": "NOT_IMPORTANT", 
                   #"disagreement": "DISAGREE",
                   #"refusal": "REFUSE",
                  #"warning": "WARN", 
                  #"threat": "THREATEN",
                  #"unwillingness": "UNWILLING",
                  #"distrust": "DISTRUST",
                  "joy, surprise or other emotion": "OTHER"}

h_ts = ['This person feels {}.', 'This person conveys {}.','This person shows {}.','This person expresses {}.', 
        'This text is {}.', 'This text is about {}.', 'This text shows {}.', 'This text expresses {}.', 'This text conveys {}.',
       'The communicative function of this text is {}.', 'The communicative intention of this text is {}.', 'The emotion of this text is {}.', 
        'The emotion expressed in this text is {}.'] # the templates used

def zero_shot_pipeline(example):
    output = classifier(example['text'], candidate_labels=list(candidate_labels.keys()), hypothesis_template=ht, multi_label=False)
    labels_scores = {candidate_labels[key]: score for key, score in zip(output['labels'], output['scores'])}
    label, score = sorted(labels_scores.items(), key=lambda kv: kv[1], reverse=True)[0]
    return {"predicted_label": label, "score": score}

In [3]:
# Use GPU if available
classifier = pipeline('zero-shot-classification', model=zero_shot_model, device=0)

for ht in h_ts:
    print(ht)
    emotion['test'] = emotion['test'].map(zero_shot_pipeline)   
    y_true = emotion['test']["function"]
    y_pred = emotion['test']["predicted_label"]
    clf_report = classification_report(
            y_true, y_pred, zero_division=0)
    print(clf_report)
    clf_report = classification_report(
            y_true, y_pred, zero_division=0, output_dict=True)
    df = pd.DataFrame(clf_report).transpose()
    with open(f"classification_report_emotion_ht={ht}_zero_shot.csv", 'w') as csv_file:
        df.to_csv(path_or_buf=csv_file)
    ds_zero_shot = emotion['test'].to_pandas()
    cols = ["text", "emotion", "function", "predicted_label", "score"]
    df_test = ds_zero_shot[:][cols]
    df_test.to_csv(f"preds_emotion_ht={ht}_zero_shot.csv",header =True, sep = '\t',index=False)

/home/njfernandez/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Parameter 'function'=<function zero_shot_pipeline at 0x7f28f777c4c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. S

This person feels {}.


  0%|          | 8/2000 [00:00<01:38, 20.23ex/s]/home/njfernandez/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [01:43<00:00, 19.39ex/s]


              precision    recall  f1-score   support

       ANGRY       0.43      0.50      0.46       275
       OTHER       0.90      0.58      0.71       920
 PESSIMISTIC       0.56      0.43      0.48       581
     WORRIED       0.27      0.77      0.40       224

    accuracy                           0.55      2000
   macro avg       0.54      0.57      0.51      2000
weighted avg       0.67      0.55      0.57      2000

This person conveys {}.


  0%|          | 0/2000 [00:00<?, ?ex/s]/home/njfernandez/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [01:43<00:00, 19.36ex/s]


...

              precision    recall  f1-score   support

       ANGRY       0.51      0.41      0.45       275
       OTHER       0.84      0.59      0.69       920
 PESSIMISTIC       0.49      0.69      0.58       581
     WORRIED       0.36      0.52      0.43       224

    accuracy                           0.59      2000
   macro avg       0.55      0.55      0.54      2000
weighted avg       0.64      0.59      0.60      2000

The emotion of this text is {}.


  0%|          | 0/2000 [00:00<?, ?ex/s]/home/njfernandez/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [01:38<00:00, 20.28ex/s]


              precision    recall  f1-score   support

       ANGRY       0.52      0.41      0.46       275
       OTHER       0.84      0.70      0.77       920
 PESSIMISTIC       0.58      0.62      0.60       581
     WORRIED       0.38      0.66      0.48       224

    accuracy                           0.64      2000
   macro avg       0.58      0.60      0.58      2000
weighted avg       0.67      0.64      0.64      2000

The emotion expressed in this text is {}.


  0%|          | 0/2000 [00:00<?, ?ex/s]/home/njfernandez/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [01:37<00:00, 20.54ex/s]

              precision    recall  f1-score   support

       ANGRY       0.54      0.41      0.47       275
       OTHER       0.83      0.72      0.77       920
 PESSIMISTIC       0.59      0.61      0.60       581
     WORRIED       0.38      0.66      0.48       224

    accuracy                           0.64      2000
   macro avg       0.58      0.60      0.58      2000
weighted avg       0.67      0.64      0.65      2000

